In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import time
import glob
from tqdm import tqdm

from ssd_model import DSOD300, DSOD512
from ssd_utils import PriorUtil
from ssd_data import InputGenerator
from ssd_training import SSDLoss, SSDFocalLoss
from utils.training import Logger, LearningRateDecay, AdamAccumulate
from utils.model import load_weights


np.set_printoptions(linewidth=120, suppress=True, precision=8)

### Data

In [ ]:
# Pascal VOC 2012 + MS COCO

from data_voc import GTUtility
gt_util_voc = GTUtility('data/VOC2012/')

from data_coco import GTUtility
gt_util_coco = GTUtility('data/COCO/')
gt_util_coco_val = GTUtility('data/COCO/', validation=True)

gt_util_train = GTUtility.merge(gt_util_voc, gt_util_coco.convert_to_voc())
gt_util_val = gt_util_coco_val.convert_to_voc()

class_weights = np.array([0.00007205, 1.3919328 , 1.43665262, 1.30902077, 1.36668928, 1.2391509 , 1.21337629, 
                          0.41527107, 1.1458096 , 0.29150119, 1.25713104, 0.61941517, 1.03175604, 1.21542005, 
                          1.01947561, 0.0542007 , 1.12664538, 1.14966073, 1.12464889, 1.49998021, 1.09218961])

### Model

In [ ]:
# DSOD 300
model = DSOD300(num_classes=gt_util_train.num_classes, softmax=False)
prior_util = PriorUtil(model)

experiment = 'dsod300fl_voccoco'
batch_size = 16

In [ ]:
# DSOD 512
model = DSOD512(num_classes=gt_util_train.num_classes, softmax=False)
prior_util = PriorUtil(model)

experiment = 'dsod512fl_voccoco'
batch_size = 6

In [ ]:
prior_util.compute_class_weights(gt_util_train)

### Training

In [ ]:
# DSOD paper
# batch size 128
# 320k iterations
# initial learning rate 0.1

epochs = 1000
initial_epoch = 0

gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size, augmentation=True)
gen_val = InputGenerator(gt_util_val, prior_util, batch_size, model.image_size, augmentation=True)

checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

#optim = keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0, nesterov=True)
#optim = keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optim = AdamAccumulate(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, accum_iters=128//batch_size) # virtual batch size 128

# weight decay
regularizer = keras.regularizers.l2(5e-4) # None if disabled
for l in model.layers:
    if l.__class__.__name__.startswith('Conv'):
        l.kernel_regularizer = regularizer

#loss = SSDLoss(alpha=1.0, neg_pos_ratio=3.0)
loss = SSDFocalLoss(lambda_conf=1.0, class_weights=class_weights)

model.compile(optimizer=optim, loss=loss.compute, metrics=loss.metrics)
        
history = model.fit_generator(
        gen_train.generate(), 
        steps_per_epoch=gen_train.num_batches,
        epochs=epochs, 
        verbose=1, 
        callbacks=[
            keras.callbacks.ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', verbose=1, save_weights_only=True),
            Logger(checkdir),
            #LearningRateDecay()
        ], 
        validation_data=gen_val.generate(), 
        validation_steps=gen_val.num_batches, 
        class_weight=None, 
        #max_queue_size=10, 
        workers=1, 
        #use_multiprocessing=False, 
        initial_epoch=initial_epoch)

### Prediction

In [ ]:
model.load_weights('./checkpoints/201808261035_dsod300fl_voccoco/weights.110.h5')

In [ ]:
_, inputs, images, data = gt_util_val.sample_random_batch(batch_size=16, input_size=model.image_size, seed=None)

In [ ]:
preds = model.predict(inputs, batch_size=1, verbose=1)

In [ ]:
for i in range(len(preds)):
    plt.figure(figsize=[8]*2)
    plt.imshow(images[i])
    res = prior_util.decode(preds[i], confidence_threshold=0.40)
    prior_util.plot_results(res, classes=gt_util_val.classes, show_labels=True, gt_data=data[i])
    plt.axis('off')
    plt.show()